In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import string
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np
import re

2025-09-30 17:10:48.142846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759252248.365052     193 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759252248.427717     193 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load data

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[0-9]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', ' ', text)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub('', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [4]:
path = "/kaggle/input/10000-vietnamese-books/output"
texts = []
limit = 0
for file_name in os.listdir(path):
    if limit < 5:
        file_path = os.path.join(path, file_name)
        with open(file_path) as file:
            for line in file:
                sentences = line.split(".")
                for sentence in sentences:
                    cleaned_sen = clean_text(sentence)
                    if cleaned_sen != "":
                        texts.append(cleaned_sen)
        limit +=1

print(f'Số lượng data {len(texts)}')

Số lượng data 9247


In [5]:
texts[:10]

['mạc can',
 'nhà ảo thuật',
 'có một cậu bé muốn học vài trò ảo thuật nhưng không tìm đâu ra trường và thầy dạy',
 'một hôm rỗi rảnh người cha chở cậu con trai nhỏ xíu khoảng tuổi ngồi háo hức sau yên xe honda',
 'vòng quanh đường phố một lúc đang bình thường thì tự nhiên chiếc xe khục khịch ho khan dừng lại đầu hẻm như là có hẹn trước ngay chóc tiệm thuốc tây và cái tủ bán thuốc lá',
 'nhưng sau cái quầy trong tiệm và trên chiếc ghế thấp nơi tủ thuốc lá đặt nhờ ngoài hè đường không có người bán hàng nào',
 'còn lạ hơn là mọi người trong đó có ông cha cố ông thích đi bộ thường ngày vẫn đi bộ và bây giờ cũng vậy đang mỉm cười đếm bước',
 'cha vẫy tay chào những người khác đang chạy xe quanh đó dù họ đi hay chạy mà tới giới hạn này tức khoảng đường có tủ thuốc lá thì ai cũng',
 'mờ đi và mất tiêu',
 'cha cố vẫn nghe người ta nói với ông rằng thỉnh thoảng cha cũng']

# Tách từ

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

In [7]:
total_words

6833

In [8]:
list(tokenizer.word_index.items())[:10]

[('và', 1),
 ('một', 2),
 ('của', 3),
 ('những', 4),
 ('có', 5),
 ('là', 6),
 ('không', 7),
 ('trong', 8),
 ('các', 9),
 ('tôi', 10)]

# Tách câu

Tách từng câu thành nhiều phần có chiều dài tăng dần để làm từng điểm dữ liệu

In [9]:
input_sequences = []
for line in texts:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [10]:
input_sequences[:10]

[[1567, 1157],
 [46, 687],
 [46, 687, 266],
 [5, 2],
 [5, 2, 1049],
 [5, 2, 1049, 732],
 [5, 2, 1049, 732, 109],
 [5, 2, 1049, 732, 109, 182],
 [5, 2, 1049, 732, 109, 182, 512],
 [5, 2, 1049, 732, 109, 182, 512, 545]]

In [11]:
tokenizer.sequences_to_texts([[3, 2, 247, 366, 83, 138]])

['của một trọng giảm mới trước']

In [12]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [13]:
input_sequences[:10]

array([[   0,    0,    0, ...,    0, 1567, 1157],
       [   0,    0,    0, ...,    0,   46,  687],
       [   0,    0,    0, ...,   46,  687,  266],
       ...,
       [   0,    0,    0, ...,  732,  109,  182],
       [   0,    0,    0, ...,  109,  182,  512],
       [   0,    0,    0, ...,  182,  512,  545]], dtype=int32)

In [14]:
# Lấy cột cuối cùng để làm nhãn
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]

In [15]:
for i in range(10):
  print("{} ---> {}".format(" ".join(tokenizer.sequences_to_texts(predictors[i:i+1])), tokenizer.sequences_to_texts([label[i:i+1]])[0]))

mạc ---> can
nhà ---> ảo
nhà ảo ---> thuật
có ---> một
có một ---> cậu
có một cậu ---> bé
có một cậu bé ---> muốn
có một cậu bé muốn ---> học
có một cậu bé muốn học ---> vài
có một cậu bé muốn học vài ---> trò


In [16]:
label = ku.to_categorical(label, num_classes=total_words)

In [17]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Xây dựng model

In [18]:
model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))

model.add(Bidirectional(LSTM(150, return_sequences = True)))

model.add(Dropout(0.2))

model.add(LSTM(100))

model.add(Dense(int(total_words / 2), activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1759252259.626933     193 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1759252259.627690     193 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
history = model.fit(predictors, label, epochs=100, batch_size=32, verbose=1)

Epoch 1/100


I0000 00:00:1759252291.805718     230 cuda_dnn.cc:529] Loaded cuDNN version 90300


7851/7851 ━━━━━━━━━━━━━━━━━━━━ 370s 46ms/step - accuracy: 0.0236 - loss: 6.7085
Epoch 2/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 360s 46ms/step - accuracy: 0.0417 - loss: 6.1967
Epoch 3/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 359s 46ms/step - accuracy: 0.0720 - loss: 5.9191
Epoch 4/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 359s 46ms/step - accuracy: 0.0980 - loss: 5.7095
Epoch 5/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 359s 46ms/step - accuracy: 0.1189 - loss: 5.5428
Epoch 6/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 359s 46ms/step - accuracy: 0.1365 - loss: 5.3880
Epoch 7/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 359s 46ms/step - accuracy: 0.1503 - loss: 5.2732
Epoch 8/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 358s 46ms/step - accuracy: 0.1622 - loss: 5.1725
Epoch 9/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 358s 46ms/step - accuracy: 0.1695 - loss: 5.1040
Epoch 10/100
7851/7851 ━━━━━━━━━━━━━━━━━━━━ 358s 46ms/step - accuracy: 0.1782 - loss: 5.0195
Epoch 11/100
6236/7851 ━━━━━━━━━━━━━━━━━━━━ 1:13 46ms/step - accuracy: 0.1798 - loss: 4.979

KeyboardInterrupt: 

In [20]:
model.save("lstm_generate_text.h5")

In [47]:
test_seq = 'cậu bé biết'

In [48]:
next_words = 10

for _ in range(next_words):
  # Chuyển câu thành vector
  token_list = tokenizer.texts_to_sequences([test_seq])[0]

  # Padding câu
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

  # Dự đoán từ tiếp theo
  predicted = model.predict(token_list, verbose=0)

  output_word = ""

  predicted_id = np.argmax(predicted)



  if predicted_id in tokenizer.index_word:
    output_word = tokenizer.index_word[predicted_id]


    if output_word == '<end>':
        break
    test_seq += " " + output_word
    print('-test_seq', test_seq)
  else:
    break

print(test_seq)

-test_seq cậu bé biết rằng
-test_seq cậu bé biết rằng họ
-test_seq cậu bé biết rằng họ có
-test_seq cậu bé biết rằng họ có thể
-test_seq cậu bé biết rằng họ có thể có
-test_seq cậu bé biết rằng họ có thể có thể
-test_seq cậu bé biết rằng họ có thể có thể có
-test_seq cậu bé biết rằng họ có thể có thể có thể
-test_seq cậu bé biết rằng họ có thể có thể có thể có
-test_seq cậu bé biết rằng họ có thể có thể có thể có thể
cậu bé biết rằng họ có thể có thể có thể có thể
